In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
import cmath
import math
import operator
from types import SimpleNamespace
import functools

import numpy as np
cot = lambda x: 1 / np.tan(x)

def nu_arg(p):
    num = p.E_z * np.exp(1j * p.theta) - 1j * p.alpha * p.k_x
    denom = p.E_z**2 + p.alpha * p.k_x * (p.alpha * p.k_x - 2 * p.E_z * np.sin(p.theta))
    return num / np.sqrt(denom)

def omega(p, j, sign):
    op = {-1: operator.sub,
          +1: operator.add}[sign]
    return 0.25 * op(gamma(p, j), delta(p, j))

def gamma(p, j):
    k = k_j(p, j)
    x = 1j * k * np.tan(p.W * k / 2)
    return (q(p) + x) / (q(p) - x)

def delta(p, j):
    k = k_j(p, j)
    x = 1j * k * cot(p.W * k / 2)
    return (q(p) - x) / (q(p) + x)

def beta(p, sign):
    w_1 = omega(p, 1, sign)
    w_2 = omega(p, 2, sign)
    nu = nu_arg(p)
    beta = np.array([
        [nu * (w_1 - w_2), (w_1 + w_2)],
        [-(w_1 + w_2), nu.conj() * (w_2 - w_1)]
    ])
    return beta

def S(p):
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    r_ll = beta(p, +1)
    t_rl = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_lr = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_rr = np.exp(-2j * q(p) * p.W) * beta(p, +1)

    S = np.block([[r_ll, t_rl],
                  [t_lr, t_rr]])
    return S

def q(p):
    return np.sqrt((2 * p.m_eff / p.hbar**2) * p.mu_n - p.k_x**2)

def k_j(p, j):
    op = {1: operator.sub,
          2: operator.add}[j]
    sqrt_term = np.sqrt(
        p.E_z**2
        - 2 * p.alpha * p.E_z * np.sin(p.theta) * p.k_x + p.alpha**2 * p.k_x**2)
    k_sq = (2 * p.m_eff / p.hbar**2) * op(p.mu_n, sqrt_term) - p.k_x**2
    return np.sqrt(k_sq)

k_F = lambda p: np.sqrt(p.mu_n * (2 * p.m_eff)) / p.hbar

In [ ]:
import scipy.constants
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (
        scipy.constants.eV * 1e-3))

params = SimpleNamespace(g=50, mu_n=10, alpha=20, B=1, theta=0, k_x=0.03, W=200, phi=np.pi, **constants)

S(params)

## kwant

In [ ]:
@functools.lru_cache()
def make_syst(a=1, W=params.W):
    import kwant
    ham = ("(0.5 * hbar**2 * (k_x**2 + k_y**2) / m_eff - mu_n) * sigma_0 + "
           "alpha * ( - k_x * sigma_y) + "
#            "alpha * (k_y * sigma_x - k_x * sigma_y) + "
           "0.5 * g * mu_B * B * (cos(theta) * sigma_x + sin(theta) * sigma_y)")
    template = kwant.continuum.discretize(ham, grid_spacing=a)
    template_sc = kwant.continuum.discretize(ham.replace('alpha', '0').replace('mu_B', '0'), grid_spacing=a)
    syst = kwant.Builder(kwant.TranslationalSymmetry([a, 0]))
    trs = 1j*sy
    lead = kwant.Builder(kwant.TranslationalSymmetry([a, 0], [0, a]), time_reversal=trs)
    syst.fill(template, lambda s: 0 <= s.pos[1] < W, (0, 0))
    syst = kwant.wraparound.wraparound(syst)
    lead.fill(template_sc, lambda s: True, (0, 0))
    lead = kwant.wraparound.wraparound(lead, keep=1)
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    return syst.finalized()
syst = make_syst()

In [ ]:
def energy_operator(syst, params):
    if isinstance(params, SimpleNamespace):
        params = dict(params.__dict__, cos=math.cos, sin=math.sin)
    
    if syst is None:
        smatrix = lambda syst, params: S(p=SimpleNamespace(**params))
    else:
        smatrix = lambda syst, params: kwant.smatrix(syst, params=params).data
    
    if params['k_x'] == 0:
        smat_plus = smat_min = smatrix(syst, params)
    else:
        params['k_x'] *= -1
        smat_min = smatrix(syst, params)
        params['k_x'] *= -1
        smat_plus = smatrix(syst, params)

    phase = cmath.exp(1j * params['phi'] / 2)
    r = np.diag([phase, phase, phase.conjugate(), phase.conjugate()])
    smat_prod = r @ smat_min.conj() @ r.conj() @ smat_plus
    return smat_prod

def energies_over_delta(syst, params):
    """Same as energy_operator(), but returns the
    square-root of the eigenvalues"""
    try:
        from numpy.lib.scimath import sqrt
        operator = energy_operator(syst, params)
        es = -1/np.linalg.eigvals(operator)
        alpha = np.hstack([sqrt(es), -sqrt(es)])
        E = alpha[alpha.imag >= 0].real
        return np.sort(E)
    except Exception:
        return np.array(4*[np.nan])

energies_over_delta(syst, params), energies_over_delta(None, params)

### Paths

In [ ]:
%%opts Path [show_legend=True]
def plot(B, alpha, mu_n, theta, phi, params=params):
    setattr(params, 'B', B)
    setattr(params, 'alpha', alpha)
    setattr(params, 'mu_n', mu_n)
    setattr(params, 'theta', theta)
    setattr(params, 'phi', phi)
    syst = make_syst(W=params.W)
    k_max = 0.92*k_F(params)
    ks = np.linspace(-k_max, k_max, 50)
    Es_ana = [energies_over_delta(None, params) for params.k_x in ks]
    Es_num = [energies_over_delta(syst, params) for params.k_x in ks]
    return (hv.Path((ks, Es_ana), label='analytics')[:, -1.1:1.1] 
            * hv.Path((ks, Es_num), label='numerics')[:, -1.1:1.1])

hv.DynamicMap(plot, kdims=['B', 'alpha', 'mu_n', 'theta', 'phi']).redim.range(
    B=(0.01, 1), alpha=(1, 100), mu_n=(10, 50), theta=(0, np.pi), phi=(0, np.pi))

### Scatter

In [ ]:
%%opts Scatter [show_legend=True] Overlay [show_legend=True]

Scatter = lambda xs, ys, size=3, color='r': hv.Overlay([hv.Scatter((xs, y)).opts(style=dict(size=size, color=color)) 
                                             for y in np.array(ys).T])

def plot(B, alpha, mu_n, theta, phi, params=params):
    setattr(params, 'B', B)
    setattr(params, 'alpha', alpha)
    setattr(params, 'mu_n', mu_n)
    setattr(params, 'theta', theta)
    setattr(params, 'phi', phi)
    syst = make_syst(W=params.W)
    k_max = 0.92*k_F(params)
    ks = np.linspace(-k_max, k_max, 50)
    Es_ana = [energies_over_delta(None, params) for params.k_x in ks]
    Es_num = [energies_over_delta(syst, params) for params.k_x in ks]  #  times 10?
    return (Scatter(ks, Es_ana, size=6, color='b')[:, -1.1:1.1].relabel('analytics')
            * Scatter(ks, Es_num)[:, -1.1:1.1].relabel('numerics'))

hv.DynamicMap(plot, kdims=['B', 'alpha', 'mu_n', 'theta', 'phi']).redim.range(
    B=(0.01, 1), alpha=(1, 100), mu_n=(10, 50), theta=(0, np.pi/4), phi=(0, np.pi))